# Data Migration 
Before using the database, logdata was storted to csv files into the `logs` directory. This Notebook creates a sqlite database `site.db` and writes the logdata to that file.

In [ ]:
import os
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
from app import db, WalliStat, Campaign

In [ ]:
db.create_all()

In [ ]:
t = dt.datetime.now()    
next_hour = t.replace(second=0, microsecond=0, minute=0, hour=t.hour+1)
hourly = Campaign(id=0, title="hourly", start=next_hour, end=dt.datetime(2050, 1, 1),
                  interval=dt.timedelta(seconds=3600))
hourly

In [ ]:
db.session.add(hourly)
db.session.commit()

In [ ]:
path_ = os.path.join("..", "logs")
for fn in os.listdir(path_):
    if fn.startswith("raw_2"):
        date_str = fn.split(".")[0].split("_")[1]
        df = pd.read_csv(os.path.join(path_, fn))
        df["datetime"] = [pd.to_datetime(date_str + " " + t).to_pydatetime() for t in df["time"]]
        dscr = df["datetime"].describe()
        print(fn, dscr["count"], dscr["first"], dscr["last"])

## Try resampling on one example logfile

In [ ]:
fn = "raw_2021-10-07.csv"
date_str = fn.split(".")[0].split("_")[1]
df = pd.read_csv(os.path.join("..", "logs", fn))
df["datetime"] = [pd.to_datetime(date_str + " " + t).to_pydatetime() for t in df["time"]]
df.head()

In [ ]:
lowres = df.set_index("datetime").resample(dt.timedelta(seconds=3600)).first()
lowres.shape

In [ ]:
plt.figure(figsize=(13,5))
plt.plot(df["datetime"], df["Temp"])
plt.plot(lowres.index, lowres["Temp"], "o")
plt.grid();

## Resample on all logfiles

In [ ]:
highres = pd.DataFrame()
path_ = os.path.join("..", "logs")
for fn in os.listdir(path_):
    if fn.startswith("raw_2"):
        print(".", end="")
        date_str = fn.split(".")[0].split("_")[1]
        df = pd.read_csv(os.path.join(path_, fn))
        df["datetime"] = [pd.to_datetime(date_str + " " + t).to_pydatetime() for t in df["time"]]
        df.drop("time", axis=1, inplace=True)
        highres = pd.concat([highres, df], ignore_index=True)
highres.shape

In [ ]:
highres.set_index("datetime", inplace=True)
highres.shape


In [ ]:
lowres = highres.resample(dt.timedelta(seconds=3600)).first()
lowres.shape

Some plausibility plots

In [ ]:
plt.figure(figsize=(20,5))
plt.plot(lowres.index, lowres["Temp"]/10., label="Temp [°C]")
plt.plot(lowres.index, lowres["P"]/1000., label="Power [KW]")
plt.grid(), plt.legend(), plt.tight_layout();

In [ ]:
lowres["P"].plot(figsize=(20,4));

In [ ]:
lowres[["V_L1", "V_L2", "V_L3"]].plot(figsize=(20,4));

In [ ]:
lowres.charge_state.value_counts()

## Write lowres data to database ``site.db``

In [ ]:
for index, row in lowres.reset_index().iterrows():
    row["campaign_id"] = 0   # hourly campaign
    ws = WalliStat.from_series(row)
    db.session.add(ws)
db.session.commit()

In [ ]:
ws